## --- Day 15: Chiton ---
What is the lowest total risk of any path from the top left to the bottom right?

In [11]:
import aoc_utils

class ChitonGraph:
    def __init__(self, filename):
        with open(filename) as f:
            raw = f.read()
        
        self.weights = aoc_utils.str_to_2d_array(raw)
        self.vertices = set()
        self.end_vertex = (len(self.weights)-1, len(self.weights[0])-1)

        for r in range(len(self.weights)):
            for c in range(len(self.weights[0])):
                self.vertices.add((r, c))

    def find_shortest_path(self):
        # Distance of every vertex from the origin (0, 0)
        distance = {v: float("inf") for v in self.vertices}

        # Start at origin, distance=0
        curr_vertex = (0, 0)
        curr_distance = 0

        while curr_vertex != self.end_vertex:
            del(distance[curr_vertex])
            r, c = curr_vertex

            # For every adjacent unvisited vertex, update distance from origin
            # (note that not all of these vertices exist)
            adjacent_vertices = [(r, c-1), (r, c+1), (r-1, c), (r+1, c)]
            for adj_v in adjacent_vertices:
                if adj_v in distance:
                    adj_r, adj_c = adj_v
                    dist_to_v = self.weights[adj_r][adj_c]
                    distance[adj_v] = min(distance[adj_v], curr_distance + dist_to_v)

            # Get the next vertex with the shortest distance
            min_dist = min(distance.values())
            curr_vertex, curr_distance =\
                next((v, d) for v, d in distance.items() if d == min_dist)
        
        return curr_distance

In [13]:
ex1_graph = ChitonGraph("./inputs/Day15ex.txt")
ex1_solution = 40

assert ex1_solution == ex1_graph.find_shortest_path()

In [14]:
p1_graph = ChitonGraph("./inputs/Day15.txt")
p1_graph.find_shortest_path()

390

## Part 2
Using the full map, what is the lowest total risk of any path from the top left to the bottom right?

((0, 0), 100)